In [ ]:
import requests
import numpy as np
import pandas as pd
import logging

In [ ]:
! python -version

### Minha API

In [ ]:
headers = {"chave-api-dados" : "insira_seu_token"}


### Acesso ao portal da transparência do governo federal playground para a criação da extração

In [ ]:
url = 'https://api.portaldatransparencia.gov.br/api-de-dados/emendas?ano=2022&pagina=2'

resposta_ = requests.get(url, headers=headers)

In [ ]:
data1 = resposta_.json()

In [ ]:
data2 = resposta_.json()

In [ ]:
url = 'https://api.portaldatransparencia.gov.br/api-de-dados/emendas?ano=2022&pagina='

In [ ]:
# Emendas página até 408

list_urls = [url + str(i) for i in range(1, 409)]

In [ ]:
list_urls[:2]

['https://api.portaldatransparencia.gov.br/api-de-dados/emendas?ano=2022&pagina=1',
 'https://api.portaldatransparencia.gov.br/api-de-dados/emendas?ano=2022&pagina=2']

In [ ]:
# passando a lista para um array
arr_urls = np.asarray(list_urls)

In [ ]:
vec_run = np.vectorize(run_request)

arr_responses = vec_run(arr_urls)

In [ ]:
arr_responses = arr_responses.tolist()

In [ ]:
arr_responses[1]

In [ ]:
reposta = np.concatenate(arr_responses).tolist()

In [ ]:
df = pd.DataFrame(reposta)

In [ ]:
df

In [ ]:
df_area = df[['funcao', 'valorEmpenhado']]
df_area['valorEmpenhado'] = df_area['valorEmpenhado'].str.replace('.', '').str.replace(',', '.').astype(np.float64)

<ipython-input-186-94971b8f1f3b>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_area['valorEmpenhado'] = df_area['valorEmpenhado'].str.replace('.', '').str.replace(',', '.').astype(np.float64)
<ipython-input-186-94971b8f1f3b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_area['valorEmpenhado'] = df_area['valorEmpenhado'].str.replace('.', '').str.replace(',', '.').astype(np.float64)


In [ ]:
df_area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6108 entries, 0 to 6107
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   funcao          6108 non-null   object 
 1   valorEmpenhado  6108 non-null   float64
dtypes: float64(1), object(1)
memory usage: 95.6+ KB


In [ ]:
agg_area = df_area.groupby('funcao').sum('valorEmpenhado')

In [ ]:
agg_area

,valorEmpenhado
funcao,
Administração,2.306679e+06
Agricultura,2.322263e+08
Assistência social,1.353689e+09
Ciência e Tecnologia,2.871393e+07
Comunicações,1.065198e+07
Comércio e serviços,4.521835e+07
Cultura,5.476948e+07
Defesa nacional,5.556213e+08
Desporto e lazer,4.118776e+08


In [ ]:
1.380082e+09/23670334710.190002


0.05830428749306528

In [ ]:
agg_area = agg_area['valorEmpenhado']/df_area['valorEmpenhado'].sum()

In [ ]:
agg_area = agg_area.reset_index()

In [ ]:
agg_area.to_csv('distribuicao_empenho_area_2022.csv')

## Criação de ETL e métodos organizados para monitoramento e automação

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [ ]:
def run_request(url_):
  resposta = requests.get(url_, headers=headers)
  return resposta.json()

In [ ]:
def coletar_dados(url):
  logging.info('Pegou URL para a criação das diferentes URLs criando uma para cada página')
  list_urls = [url + str(i) for i in range(1, 6)]
  arr_urls = np.asarray(list_urls)

  vec_run = np.vectorize(run_request)
  logging.info('Fazendo uma requisição para cada URL')
  arr_responses = vec_run(arr_urls)
  logging.info('Requisições concluídas')
  arr_responses = arr_responses.tolist()
  reposta = np.concatenate(arr_responses).tolist()
  return reposta

In [ ]:
def transformar_dado(dado):
  logging.info('Criando dataframe')
  # Insira o código para a criação do dataframe a partir do parâmetro 'dado'
  # A variável deve ter o nome df_area


  # Insita o log sobre o que está sendo executado, utilizando 'logging.info'

  df_area = df[['funcao', 'valorEmpenhado']]

  # Insita o log sobre o que está sendo executado, utilizando 'logging.info'
  df_area['valorEmpenhado'] = df_area['valorEmpenhado'].str.replace('.', '').str.replace(',', '.').astype(np.float64)

  logging.info('Transformando dado string -> float')
  agg_area = df_area.groupby('funcao').sum('valorEmpenhado')

  logging.info('Transformando dados para retirar proporção em porcentagem')
  agg_area = agg_area['valorEmpenhado']/df_area['valorEmpenhado'].sum()

  # Insira o código para reset de index de agg_area, usando reset_index()

  return agg_area

In [ ]:
def carga(dado):
  # crie um arquivo CSV com p parâmetro 'dado', o nome do arquivo deve ser 'distribuicao_empenho_area_20220.csv'
  # esse método não tem retorno


In [ ]:
# Escreva aqui quais são os passos para a extração dos dados e processamento
# Este método não tem retorno, no final será gerado um CSV com os dados a serem salvos

def etl():
  url = 'https://api.portaldatransparencia.gov.br/api-de-dados/emendas?ano=2022&pagina='
  # colete os dados
  # transforme os dados
  # salve os dados


In [ ]:
etl()